![logo](./img/logo-onera.png "Logo ONERA") ![logo](./img/logo-ISAE_SUPAERO.png "Logo ISAE-SUPAERO") 

# FAST-OAD Tutorial

FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Installation and import test
In order to use this tutorial notebook, the *fastoad* package must be installed.

### Installation
FAST requests at least Python 3.7 to be used. As of today, the package is available locally but will be in the future available on [PyPi](https://pypi.org/) to make it available through a simple *pip*.
To install locally *fastoad* you must:
1. Download the project **.zip** on the [Github repository](https://github.com/fast-aircraft-design/FAST-OAD)
2. Unzip the project file
3. In a terminal run:

    `cd FAST-OAD-master`
    
    `pip install .`

### Installation Test
Run the following cell to attempt an import of the package and check if it is properly installed.

In [8]:
import fastoad

## 2. Working with user files

The FAST-OAD environment uses two types of user files:

* Configuration files (**.toml**):

    The configuration example file used for the tutorial can be found [here](./data/oad_process.toml).

    FastProblem instances require a configuration file. The configuration file uses the .toml syntax. In this file the user defines:

    1 - Path to search disciplinary modules
```toml
module_folders = []
```


    The actual path is empty since the native modules of FAST-OAD are already made available. However, if the user wants to integrate his own modules he must specify the path to the folder that contains the modules.

    2 - Inputs and outputs filenames
```toml
input_file = "../outputs/oad_process_inputs.xml"
output_file = "../outputs/oad_process_outputs.xml"
```


    The input file contains the global inputs values required to run all the model.
    The output file contains all the variables (inputs + outputs) values obtained after a model run.

    3 - The system level structure of the model
```toml
[model]
    nonlinear_solver = "om.NonlinearBlockGS(maxiter=100, iprint=0)"
    linear_solver = "om.ScipyKrylov()"
    [model.geometry]
        id = "fastoad.geometry.legacy"
    [model.aerodynamics]
        id = "fastoad.aerodynamics.highspeed.legacy"
    [model.performance_loop]
        nonlinear_solver = "om.NonlinearBlockGS(iprint=0)"
        [model.performance_loop.performance]
            id = "fastoad.performances.breguet.implicit"
        [model.performance_loop.sizing_loop.propulsion]
            id = "fastoad.propulsion.rubber_engine"
```


    4 - The parameters of the optimization problem

* System variables files (**.xml**):

    These file regroup the information of the variables involved in the system model. There are two types of system variables files:
    
    1 - INPUT_FILE: The input file contains the global inputs values required to run all the model. The user is free to modify the values of the variables in order that these new values are considered during a model run.
    
    2 - OUTPUT_FILE: The output file contains all the variables (inputs + outputs) values obtained after a model run.
    

![User Files](./img/user_files_arch.svg "User Files Architecture")

In [1]:
import os.path as pth
import openmdao.api as om
from fastoad.cmd import api

DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.toml')
REF_VALUES_FILE = pth.join(DATA_FOLDER_PATH, 'CeRAS01_baseline.xml')


Unable to import mpi4py. Parallel processing unavailable.


In [2]:
api.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

In [3]:
api.generate_inputs(CONFIGURATION_FILE, True, REF_VALUES_FILE, 'legacy')

The xpath Aircraft/propulsion/conventional/k_factors/k_SFC does not have any variable affected in the translator.
The xpath Aircraft/propulsion/conventional/k_factors/k_corr does not have any variable affected in the translator.
The xpath Aircraft/propulsion/conventional/k_factors/k_co2 does not have any variable affected in the translator.


In [4]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
print(N2_FILE)
IFrame(src=N2_FILE, width='100%', height='500px')

workdir\n2.html


## 3. Running your first MDA

In [5]:
# problem.set_solver_print(level=2)
api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

D:\cdavid\PyCharmProjects\FAST-OAD\.venv\lib\site-packages\openmdao\core\group.py:1100: UserWarning:Group (<model>): Input 'weights.weights.owe.flight_controls_weight.weight:airframe:flight_controls:mass:offset' with units of 'kg' is connected to output 'inputs.weight:airframe:flight_controls:mass:offset' which has no units.
D:\cdavid\PyCharmProjects\FAST-OAD\.venv\lib\site-packages\openmdao\core\group.py:1100: UserWarning:Group (<model>): Input 'weights.weights.owe.wing_weight.weight:airframe:wing:mass:offset' with units of 'kg' is connected to output 'inputs.weight:airframe:wing:mass:offset' which has no units.
D:\cdavid\PyCharmProjects\FAST-OAD\.venv\lib\site-packages\openmdao\core\group.py:1100: UserWarning:Group (<model>): Input 'weights.weights.owe.wing_weight.weight:airframe:wing:bending_sizing:mass:offset' with units of 'kg' is connected to output 'inputs.weight:airframe:wing:bending_sizing:mass:offset' which has no units.
D:\cdavid\PyCharmProjects\FAST-OAD\.venv\lib\site-packa

|  
|  ==========
|  perfo_loop
|  ==========


d:\cdavid\pycharmprojects\fast-oad\src\fastoad\modules\performances\breguet.py:235: RuntimeWarning:

overflow encountered in exp

MAX THRUST computation for T4 above 1600.0 may be unreliable.
SFC RATIO computation for thrust rate above 1.0 may be unreliable.


|  NL: NLBGSSolver 'NL: NLBGS' on system 'perfo_loop' failed to converge in 10 iterations.


MAX THRUST computation for T4 above 1600.0 may be unreliable.
SFC RATIO computation for thrust rate above 1.0 may be unreliable.


...

|  
|  ==========
|  perfo_loop
|  ==========
|  NL: NLBGS Converged in 1 iterations


MAX THRUST computation for T4 above 1600.0 may be unreliable.


|  
|  ==========
|  perfo_loop
|  ==========
|  NL: NLBGS Converged in 1 iterations


MAX THRUST computation for T4 above 1600.0 may be unreliable.


|  
|  ==========
|  perfo_loop
|  ==========
|  NL: NLBGS Converged in 1 iterations
NL: NLBGS Converged in 11 iterations


## 4. Running your first MDO

In [6]:
# problem.set_solver_print(level=2)
api.optimize_problem(CONFIGURATION_FILE, overwrite=True)

|  
|  ==========
|  perfo_loop
|  ==========


MAX THRUST computation for T4 above 1600.0 may be unreliable.
SFC RATIO computation for thrust rate above 1.0 may be unreliable.


|  NL: NLBGSSolver 'NL: NLBGS' on system 'perfo_loop' failed to converge in 10 iterations.


MAX THRUST computation for T4 above 1600.0 may be unreliable.
SFC RATIO computation for thrust rate above 1.0 may be unreliable.


|  
|  ==========
|  perfo_loop
|  ==========
|  NL: NLBGS Converged in 1 iterations


...

|  
|  ==========
|  perfo_loop
|  ==========
|  NL: NLBGS Converged in 1 iterations


MAX THRUST computation for T4 above 1600.0 may be unreliable.


|  
|  ==========
|  perfo_loop
|  ==========
|  NL: NLBGS Converged in 1 iterations


MAX THRUST computation for T4 above 1600.0 may be unreliable.


|  
|  ==========
|  perfo_loop
|  ==========
|  NL: NLBGS Converged in 1 iterations
NL: NLBGS Converged in 11 iterations
